In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Install Google Gen AI SDK for Python

Gemini 3 API features require Gen AI SDK for Python version 1.51.0 or later.

In [ ]:
%pip install --upgrade --quiet google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.10 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.1 which is incompatible.
google-cloud-aiplatform 1.74.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.1 which is incompatible.
google-cloud-bigtable 2.27.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.1 w

### Import libraries

In [ ]:
import os
import sys

from IPython.display import HTML, Markdown, display
from google import genai
from google.genai import types
from pydantic import BaseModel

### Authenticate your Google Cloud Project for Vertex AI

You can use a Google Cloud Project or an API Key for authentication. This notebook uses a Google Cloud Project.

- [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)

In [ ]:
# fmt: off
PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global"

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Choose a Gemini 3 Pro model

Use `gemini-3-pro-preview` in this tutorial. Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [ ]:
MODEL_ID = "gemini-3-pro-preview"  # @param ["gemini-3-pro-preview"] {type: "string"}

In [ ]:
prompt = """
Act as a Strategic Assistant. Analyze this handwritten daily plan

1. Morning Briefing & Status Update: Identify the core objective and current
   status based on visual cues (e.g., icons, emphasis markers).
2. The Priority Stack:: List fixed time blocks, identifying any hard
   deadlines and tentative placeholders.
3. Productivity Guardrails: Identify and interpret  Global
   Execution Rules or time boundaries.
4. Real-time Interventions: Identify unplanned disruptions or incoming queries
    and assess their impact on the schedule.
5. Actionable Triggers: Map symbols (?, !!, ⚠) to specific tasks that
   require external data or represent critical risks.

Output the result as a 'Strategic Daily Briefing' that is concise and
action-oriented. Make sure you have covered all the points
"""

### ✅ Note

- If your content is stored in [Google Cloud Storage](https://cloud.google.com/storage), you can use the `from_uri`  method to create a `Part` object.
- If your content is stored in your local file system, you can read it in as bytes data and use the `from_bytes` method to create a `Part` object.


```
# Download and open an image locally.
! wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/meal.png

with open("meal.png", "rb") as f:
    image = f.read()

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        types.Part.from_bytes(data=image, mime_type="image/png"),
        "Write a short and engaging blog post based on this picture.",
    ],
)

display(Markdown(response.text))
```



In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    config=types.GenerateContentConfig(
        temperature=1.0,
        top_p=0.9,
        max_output_tokens=8000,
        seed=42
    ),
    contents=[
        types.Part(
            file_data=types.FileData(
                file_uri="gs://ai_camp_demo_artefacts/tasklist_1.jpg",
                mime_type="image/jpeg",
            ),
            media_resolution=types.PartMediaResolution(
                level=types.PartMediaResolutionLevel.MEDIA_RESOLUTION_HIGH,
            ),
        ),
        prompt
    ],
)

display(Markdown(response.text))

Based on the handwritten analysis of the daily plan for Wednesday 01/14, here is your **Strategic Daily Briefing**.

### 1. Morning Briefing & Status Update
*   **Core Objective:** Execute the **2:00 PM Design Review**. This is the anchor event of the day; success depends entirely on completing the presentation deck.
*   **Current Status:** **CRITICAL / AT RISK**. The visual marker **"⚠ running behind"** combined with **"Finish slides !!!"** indicates immediate schedule compression. The note "push non-urgent?" suggests you are already in triage mode.

### 2. The Priority Stack
**Fixed Time Blocks:**
*   **09:30 – 10:00:** Standup
*   **11:00 – 11:30:** 1:1 Meeting
*   **14:00 – 15:00:** **Design Review (High Stakes)**

**Hard Deadlines:**
*   **13:30:** Slides must be finished (30-minute buffer before review).
*   **EOD:** Reply to John Doe regarding metrics.

**Tentative/Placeholders:**
*   **16:30:** Catch-up (Marked "Tentative" & Circled).
*   **Gym:** (Marked with "?") – Likely to be cut given the schedule compression.

### 3. Productivity Guardrails
*   **Deep Work Mandate:** You explicitly noted a need for a **"30 min deep focus block"** to improve morning focus. *Recommendation: Schedule this immediately between 11:30 and 13:30 to finish slides.*
*   **Boundary Conflict:** You set a rule for **"no meetings after 4 pm"**, yet have a tentative **4:30 Catch-up**. *Action: Cancel or reschedule the 4:30 to honor the boundary.*
*   **Prerequisite:** **"COFFEE FIRST"** (Non-negotiable activation step).

### 4. Real-time Interventions
Two unplanned disruptions have entered the workflow (scribbled in colored ink) that threaten your slide deadline:
*   **Red Ink (High Risk):** "Product team asking for quick input?" -> *Strategy: Defer response until after 2 PM.*
*   **Blue Ink (Medium Risk):** "Customer question came in?" -> *Strategy: Route to "Email backlog" slot; do not process now.*

### 5. Actionable Triggers
*   **!!! (Slides):** **Immediate Action Required.** Abandon "Email backlog" tasks until slides are complete.
*   **? (John Doe/Metrics):** **Data Gap.** You cannot reply to John without the metrics. *Action: Ping data source immediately so numbers are ready for your EOD reply.*
*   **⚠ (Running Behind):** **Contingency Trigger.** Execute your note to "push non-urgent."
*   **Boxed (CALL MOM):** **Personal Priority.** High visibility marker suggests this cannot slip to tomorrow despite work stress.
*   **Jane Note:** "Remember to ask Jane about timelines." *Action: Add to 9:30 Standup agenda.*

In [ ]:
image_generation = client.models.generate_content(
    model="gemini-3-pro-image-preview",
    contents=f"""
    Generate an infographic of the daily schedule: {response.text}.
    Stick to the facts and do not miss any steps
    """,
    config=types.GenerateContentConfig(
        image_config=types.ImageConfig(
            aspect_ratio="16:9",
            image_size="4K"
        )
    )
)

image_parts = [part for part in image_generation.parts if part.inline_data]

if image_parts:
    image = image_parts[0].as_image()
    image.save('infographic_schedule_1.png')
    image.show()

### Take Home Task
What if you use low resolution?
What variation do you see in the response?

In [ ]:
# response = client.models.generate_content(
#     model=MODEL_ID,
#     contents=[
#         types.Part(
#             file_data=types.FileData(
#                 file_uri="gs://ai_camp_demo_artefacts/tasklist.jpg",
#                 mime_type="image/jpeg",
#             ),
#             media_resolution=types.PartMediaResolution(
#                 level=types.PartMediaResolutionLevel.MEDIA_RESOLUTION_LOW
#             ),
#         ),
#         prompt
#     ],
# )

# display(Markdown(response.text))

In [ ]:
prompt_1 = """
Look at the red-ink interruption and the 'Customer Questions' note. Based on the 1:30 PM slide deadline,
what is the latest possible time I can address these without missing my 2:00 PM review?
"""


In [ ]:
reasoning_response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        types.Part(
            file_data=types.FileData(
                file_uri="gs://ai_camp_demo_artefacts/tasklist_1.jpg",
                mime_type="image/jpeg",
            ),
            media_resolution=types.PartMediaResolution(
                level=types.PartMediaResolutionLevel.MEDIA_RESOLUTION_HIGH
            ),
        ),
        prompt_1
    ],
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts = True,
            thinking_level=types.ThinkingLevel.HIGH  # Dynamic thinking for high reasoning tasks
        )
    ),
)

Show thoughts and response

In [ ]:
for part in reasoning_response.candidates[0].content.parts:
    if part.thought:
        display(
            Markdown(
                f"""## Thoughts:
         {part.text}
        """
            )
        )
    else:
        display(
            Markdown(
                f"""## Answer:
         {part.text}
        """
            )
        )

## Thoughts:
         **Refining The Recommendation**

\n\n

I've clarified the constraints and critical path. My analysis highlighted the 1:30 PM slide deadline as the key driver. The initial thought was to use the 1:30 PM to 2:00 PM window. However, the "running behind" note and the urgent "finish slides!!!" message solidified that it's safer to postpone these tasks until *after* the slides are done at 1:30 PM. The final answer: 1:30 PM, immediately after the slide deadline. The response now clearly reflects this.



        

## Answer:
         Based on the schedule and notes in the image, the latest possible time (and arguably the *safest* time) to address these interruptions is **1:30 PM**.

Here is the breakdown of the timeline:

1.  **The Priority:** You have a hard deadline noted as **"Slides by 1:30 pm"**. The note in the bottom right corner says **"running behind,"** which suggests you cannot afford to stop working on the slides before they are finished.
2.  **The Window:** Once you finish the slides at **1:30 PM**, you have a 30-minute gap before your next hard commitment.
3.  **The Hard Stop:** Your **Design Review** begins promptly at **2:00 PM**.

Therefore, you have the window between **1:30 PM and 2:00 PM** to handle the red-ink "Project team" request and the blue-ink "Customer question."
        

In [ ]:
image_generation = client.models.generate_content(
    model="gemini-3-pro-image-preview",
    contents=f"Generate an infographic of the daily schedule: {reasoning_response.text}.",
    config=types.GenerateContentConfig(
        image_config=types.ImageConfig(
            aspect_ratio="16:9",
            image_size="4K"
        )
    )
)

image_parts = [part for part in image_generation.parts if part.inline_data]

if image_parts:
    image = image_parts[0].as_image()
    image.save('infographic_reason_1.png')
    image.show()